# Taxi Travel Data Analysis

In this demo, we will be doing some demos on temporal feature engineering with the Kaggle Dataset

### Loading libraries, datasets

In [294]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
import torch

In [295]:
# These are all of the files you are given
#df_tr = pd.read_csv("archive/train.csv")


In [296]:
#df_tr.head()
# avg = df_tr['LEN'].mean()
# print(avg)

### Get Computed Time from POLYLINE

Our goal is to predict the travel-time of the taxi, which can be derived from the POLYLINE length.

Recall:

```
The travel time of the trip (the prediction target of this project) is defined as the (number of points-1) x 15 seconds. 
For example, a trip with 101 data points in POLYLINE has a length of (101-1) * 15 = 1500 seconds. Some trips have missing 
data points in POLYLINE, indicated by MISSING_DATA column, and it is part of the challenge how you utilize this knowledge.
```

We are not doing anything with the MISSING_DATA. It is up to you to find a way to use (or ignore) that information.

In [297]:
# Over every single 
def polyline_to_trip_duration(polyline):
  return max(polyline.count("[") - 2, 0) * 15

# This code creates a new column, "LEN", in our dataframe. The value is
# the (polyline_length - 1) * 15, where polyline_length = count("[") - 1
# df_tr["LEN"] = df_tr["POLYLINE"].apply(polyline_to_trip_duration)

In [298]:
from datetime import datetime
def parse_time(x):
  # We are using python's builtin datetime library
  # https://docs.python.org/3/library/datetime.html#datetime.date.fromtimestamp

  # Each x is essentially a 1 row, 1 column pandas Series
  dt = datetime.fromtimestamp(x["TIMESTAMP"])
  return dt.year, dt.month, dt.day, dt.hour, dt.weekday()

# Because we are assigning multiple values at a time, we need to "expand" our computed (year, month, day, hour, weekday) tuples on 
# the column axis, or axis 1
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html
# df_tr[["YR", "MON", "DAY", "HR", "WK"]] = df_tr[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")
def parse_midnight_minutes(x):
    dt = datetime.fromtimestamp(x["TIMESTAMP"])
    return dt.hour * 60 + dt.minute

In [299]:
def onehot(x):
    x=x.values[0]
    if x=="A":
        return 1, 0, 0
    elif x=="B":
        return 0, 1, 0
    elif x=="C":
        return 0, 0, 1
    else:
        return 0, 0, 0
    
# df_tr[["CALL_A", "CALL_B", "CALL_C"]] = df_tr[["CALL_TYPE"]].apply(onehot, axis=1, result_type="expand")   
# df_tr[["DAY_A", "DAY_B", "DAY_C"]] = df_tr[["DAY_TYPE"]].apply(onehot, axis=1, result_type="expand")     


### Create a Prediction File

In [300]:
# vals = df_tr["HR"].value_counts()
# print(vals)
# def reduce(x):
#     return x - 2013
# hr_oh = torch.nn.functional.one_hot(torch.tensor(df_tr["HR"].values))
# print(hr_oh)
# print(hr_oh.shape)

In [301]:
# vals = df_tr["TAXI_ID"].values
# print(vals)
# print(min(vals))
# print(max(vals))

# # df_tr["TAXI_ID"] = df_tr["TAXI_ID"].apply(reduce)
# vals = df_tr["TAXI_ID"].values
# print(vals)
# print(min(vals))
# print(max(vals))
# oh = torch.nn.functional.one_hot(torch.tensor(df_tr["TAXI_ID"].values))
# print(oh)
# print(oh.shape)


In [302]:
# df_tr[["CALL_A", "LEN"]].values

In [303]:
# Feature Trimming
# mean, std = df_tr["LEN"].mean(), df_tr["LEN"].std()
# median = df_tr["LEN"].median()
# outlier_threshold = 3
# df_trimmed = df_tr[df_tr["LEN"] < mean + outlier_threshold * std]
# df_trimmed = df_trimmed[df_trimmed['MISSING_DATA'] == False]
# print("Before Trimming: " + str(len(df_tr)))
# print("After Trimming: " + str(len(df_trimmed)))

# df_tr = df_trimmed

In [304]:
df_tr = pd.read_csv("archive/train.csv")
df_tr["LEN"] = df_tr["POLYLINE"].apply(polyline_to_trip_duration)
mean, std = df_tr["LEN"].mean(), df_tr["LEN"].std()
outlier_threshold = 3
df_trimmed = df_tr[df_tr["LEN"] < mean + outlier_threshold * std]
df_trimmed = df_trimmed[df_trimmed['MISSING_DATA'] == False]
df_trimmed = df_trimmed[df_trimmed['LEN'] != 0]
print("Before Trimming: " + str(len(df_tr)))
print("After Trimming: " + str(len(df_trimmed)))
df_tr = df_trimmed



trlen = len(df_tr)

df_ts = pd.read_csv("archive/test_public.csv")
df_ts["POLYLINE"]="trololololo"



df_both = pd.concat([df_tr, df_ts])
df_both["LEN"] = df_both["POLYLINE"].apply(polyline_to_trip_duration)
ocvc = df_both["ORIGIN_CALL"].value_counts()
def filterOC(x):
    if pd.isnull(x):
        return x
    if(ocvc[x] < 50):
        return None
    return x
df_both["ORIGIN_CALL"] = df_both["ORIGIN_CALL"].apply(filterOC)
print(df_both["ORIGIN_CALL"].value_counts())
# # df_tr[["YR", "MON", "DAY", "HR", "WK"]] = df_tr[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")
df_both["MIDMINS"] = df_both[["TIMESTAMP"]].apply(parse_midnight_minutes, axis=1, result_type="expand")
#df_tr["MIDMINS"]=(df_tr["MIDMINS"]-df_tr["MIDMINS"].min())/(df_tr["MIDMINS"].max()-df_tr["MIDMINS"].min())
df_both = pd.get_dummies(data=df_both, columns=['CALL_TYPE', "ORIGIN_STAND", "ORIGIN_CALL"])


df_tr = df_both.iloc[:trlen]
df_ts = df_both.iloc[trlen:]






Before Trimming: 1710670
After Trimming: 1656255
ORIGIN_CALL
2002.0     56688
63882.0     6320
2001.0      2397
13168.0     1290
6728.0      1089
           ...  
18954.0       50
15826.0       50
8110.0        50
5093.0        50
2983.0        50
Name: count, Length: 734, dtype: int64


In [309]:

class MyDataset(Dataset):
  def __init__(self, df):

    boolcols = list(df.columns)
    badcols = ["TRIP_ID", 'MIDMINS',  'TIMESTAMP', 'MISSING_DATA', 'POLYLINE', 'LEN', "TAXI_ID", "DAY_TYPE"] 
    for b in badcols:
      boolcols.remove(b)
    # print(boolcols)
    
    
         
    boolz=df[boolcols].values
    intz = df["MIDMINS"].values
    # print(boolz)
    y=df["LEN"].values

    booltens = torch.tensor(boolz,dtype=torch.float32)
    inttens = torch.tensor(intz,dtype=torch.float32).unsqueeze(1)

    # print(booltens.shape)
    # print(inttens.shape)
 
    self.x_train=torch.cat([booltens, inttens], dim=1)
    # print(self.x_train)
    self.y_train=torch.tensor(y,dtype=torch.float32)
    self.df = df
 
  def __len__(self):
    return len(self.y_train)
   
  def __getitem__(self,idx):
    return self.x_train[idx],self.y_train[idx]

In [310]:
from torch.utils.data import random_split
trds=MyDataset(df_tr)
trds, valds = random_split(trds, [0.9, 0.1])
print(len(trds), len(valds))
train_loader=DataLoader(trds,batch_size=256, shuffle=True)
val_loader=DataLoader(valds)

1490630 165625


In [311]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten(start_dim=1)
        # self.conv1 = nn.Conv2d(3, 6, 5)
        # self.pool = nn.MaxPool2d(2, 2)
        # self.conv2 = nn.Conv2d(6, 16, 5)
        self.dro = nn.Dropout(p=0.25)
        self.fc1 = nn.Linear(801, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 1)

    def forward(self, x):

        
        x = F.relu(self.fc1(x))
        x = self.dro(x)
        x = F.relu(self.fc2(x))
        # x = self.dro(x)
        x = F.relu(self.fc3(x))
        # x = self.dro(x)
        x = self.fc4(x)
        # x = torch.transpose(x, 0, 1)
        return x


net = Net()

In [312]:
import torch.optim as optim
def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))
criterion = RMSELoss
optimizer = optim.Adam(net.parameters(), lr=0.00001)

In [313]:
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        labels=labels.unsqueeze(1)
        outputs = net(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            #print(outputs[0], labels[0])
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

KeyboardInterrupt: 

In [314]:
with torch.no_grad():
    total_loss = 0.0
    for i, data in enumerate(val_loader, 0):

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data


        # forward + backward + optimize
        labels=labels
        outputs = net(inputs)
        
        loss = criterion(outputs, labels)


        # print statistics
        total_loss += loss.item()
        # if i%1000==0:
        #     print(round(loss.item()), outputs, labels)

print(f'Total Loss: {total_loss / len(val_loader):.3f}')


Total Loss: 635.196


In [315]:
testset = MyDataset(df_ts)
testloader = DataLoader(testset)
preds = []
with torch.no_grad():
    for data in testloader:
        features, labels = data
        # calculate outputs by running images through the network
        output = net(features)
        preds.append(output.item())
ids = testset.df["TRIP_ID"]        
print(preds)
d = {"TRIP_ID" : ids, "TRAVEL_TIME" : preds}
newdf = pd.DataFrame(d)
print(newdf)
newdf.to_csv("my_pred.csv", index=None)

[47.43415832519531, 48.91222381591797, 55.246734619140625, 50.59748077392578, 64.25698852539062, 56.9205207824707, 55.619293212890625, 44.270263671875, 56.29725646972656, 52.418270111083984, 57.08964920043945, 58.451297760009766, 61.36245346069336, 44.7822265625, 57.559452056884766, 52.52726745605469, 60.672210693359375, 48.79900360107422, 56.55269241333008, 54.90105056762695, 57.927001953125, 50.48965835571289, 55.6378059387207, 49.111324310302734, 65.85307312011719, 40.00066375732422, 53.19807434082031, 43.28772735595703, 61.94066619873047, 61.494056701660156, 47.66448211669922, 52.27593994140625, 61.961448669433594, 55.30530548095703, 57.578468322753906, 52.722625732421875, 60.225440979003906, 46.24495315551758, 57.22691345214844, 47.36805725097656, 46.23332977294922, 49.342708587646484, 45.1513557434082, 58.308311462402344, 58.46783447265625, 56.09163284301758, 42.48173141479492, 53.95802688598633, 43.48570251464844, 49.6137580871582, 44.46013641357422, 47.69865036010742, 63.159996

In [316]:
# # Sample submission file that is given on kaggle
# df_sample = pd.read_csv("archive/sampleSubmission.csv")

# df_sample["TRAVEL_TIME"] = 716.43

# # mean(716.43) -> 792.73593
# # median(600) -> 784.74219
# df_sample.to_csv("my_pred.csv", index=None)

**Boosted Decision Trees**


In [ ]:
def processCSV(csvname):
  xg_train = pd.read_csv(csvname)
  df_tr = xg_train
  if "POLYLINE" not in df_tr: #test dataset
    df_tr["POLYLINE"]="trololololo"
    df_tr["LEN"] = df_tr["POLYLINE"].apply(polyline_to_trip_duration)
  else: 
    df_tr["LEN"] = df_tr["POLYLINE"].apply(polyline_to_trip_duration)
    mean, std = df_tr["LEN"].mean(), df_tr["LEN"].std()
    median = df_tr["LEN"].median()
    outlier_threshold = 3
    print(mean, std)
    df_trimmed = df_tr[df_tr["LEN"] < mean + outlier_threshold * std]
    df_trimmed = df_trimmed[df_trimmed['MISSING_DATA'] == False]
    print("Before Trimming: " + str(len(df_tr)))
    print("After Trimming: " + str(len(df_trimmed)))
    df_tr = df_trimmed
    # first trim the dataset

  df_tr[["YR", "MON", "DAY", "HR", "WK"]] = df_tr[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")
  LetterToIndex = {'A': 0, 'B': 1, 'C': 2}
  df_trimmed_copy = df_tr
  print('there was ' + str(len(df_trimmed_copy['LEN'] == 0)) + " zeroes")
  df_trimmed_copy = df_trimmed_copy[df_trimmed_copy['LEN'] != 0]

  y_train = df_trimmed_copy["LEN"]
  df_trimmed_copy["CALL_TYPE"] = df_trimmed_copy["CALL_TYPE"].map(LetterToIndex)
  # df_trimmed_copy["DAY_TYPE"] = df_trimmed_copy["DAY_TYPE"].map(LetterToIndex)
  #took out "TRIP_ID"
  x_train = df_trimmed_copy[["TRIP_ID", "CALL_TYPE", "ORIGIN_CALL", "ORIGIN_STAND", "TAXI_ID", "YR", "MON", "DAY", "HR", "WK"]]
  return x_train, y_train

In [ ]:
x_train, y_train = processCSV('archive/train.csv')


In [ ]:
x_train.drop(columns=['TRIP_ID'])#this doesnt actually drop, just without tripid

In [ ]:
y_train

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


# shuffle_indices = np.random.permutation(len(x_train))
# X_shuffled = x_train[shuffle_indices]
# y_shuffled = y_train[shuffle_indices]

combined_df = pd.concat([x_train, y_train], axis=1)

# Get the number of rows in the DataFrame
num_rows = combined_df.shape[0]

# Generate a random permutation of indices
perm = np.random.permutation(num_rows)

# Shuffle the combined DataFrame using the permutation
shuffled_df = combined_df.iloc[perm]

# Split the shuffled DataFrame back into separate DataFrames
shuffled_train_df = shuffled_df.iloc[:, :-1]
shuffled_label_df = shuffled_df.iloc[:, -1]

# print(shuffled_X)
# print(shuffled_Y)

X_train, X_val, Y_train, Y_val = train_test_split(shuffled_train_df, shuffled_label_df, test_size=0.3, random_state=None)

# print(X_train, X_val, Y_train, Y_val)
# parameters = {
#     'objective': 'reg:squarederror',  # Regression objective
#     'eta': 0.1,                       # Learning rate
#     'max_depth': 3,                   # Maximum depth of each tree
#     'subsample': 0.8,                 # Subsample ratio of the training instances
#     'colsample_bytree': 0.8,          # Subsample ratio of columns when constructing each tree
#     'eval_metric': 'rmse',             # Evaluation metric to use (RMSE)
#     'shuffle': True
# }

# # Create the XGBoost regressor
# xgmodel = xgb.XGBRegressor(parameters)

# # Train the model
# xgmodel.fit(x_train, y_train)

# Assuming you have a DataFrame called 'df' with features and target variables

# Convert DataFrame to DMatrix
data_matrix = xgb.DMatrix(data = X_train.drop(columns=['TRIP_ID']), label=Y_train)
validation_matrix = xgb.DMatrix(data = X_val.drop(columns=['TRIP_ID']), label=Y_val)

# Define XGBoost parameters
# parameters = {'objective': 'reg:squarederror', 
#               'max_depth': 30, 
#               # 'booster': 'gbtree', 
#                'colsample_bytree': .6,
#                'eta': .4,
#               # 'max_depth': 5,
#               'reg_alpha': 608.85614181114505,
#              'reg_lambda': 809.260067417192285,
#               'subsample': 0.9
#               }

# parameters = {'objective': 'reg:squarederror', 'max_depth': 30, 'booster': 'gbtree', 'subsample': 0.9}
# parameters = {'objective': 'reg:squarederror', 
#               'max_depth': 4, 
#               # 'booster': 'gbtree', 
#               # 'colsample_bytree': .13641457979856397,
#               # 'max_depth': 5,
#               'reg_alpha': 68.85614181114505,
#             #   'reg_lambda': 9.260067417192285,
#               # 'subsample': 0.8980218163372579
# }

parameters = {
    'objective': 'reg:squarederror',
    'colsample_bytree': 0.5586586280723452,
    'learning_rate': 0.06889888561468978,
    'max_depth': 5,
    # 'n_estimators': 1164,
    'reg_alpha': 25.287638310615133,
    'reg_lambda': 67.51702941180568,
    'subsample': 0.77179392553171
}

xg_model = xgb.train(parameters, data_matrix, num_boost_round=10000, early_stopping_rounds=20, evals=[(validation_matrix, 'validation')], verbose_eval=100)



# ('colsample_bytree', 0.13641457979856397),
#              ('learning_rate', 0.09604083138419779),
#              ('max_depth', 5),
#              ('n_estimators', 2206),
#              ('reg_alpha', 68.85614181114505),
#              ('reg_lambda', 9.260067417192285),
#              ('subsample', 0.8980218163372579)

# reg = XGBRegressor(random_state=0, booster='gbtree', objective='reg:squarederror', tree_method='gpu_hist', **best_params)

In [ ]:
test_x, test_y = processCSV('archive/test_public.csv')
test_x

In [ ]:
y_pred = xg_model.predict(xgb.DMatrix(test_x.drop(columns=['TRIP_ID'])))


In [ ]:
ids = test_x["TRIP_ID"]
d = {"TRIP_ID" : ids, "TRAVEL_TIME" : y_pred}
newdf = pd.DataFrame(d)
print(newdf)
newdf.to_csv("tree_pred9_bayesian_opt_max_depth_5_6232_boosts.csv", index=None)


### Do some Feature Analysis

For our feature analysis, we are looking at which of our engineered features may be useful in making a taxicab time regression model

In [317]:
# First n samples to analyze. Set to -1 to use all data
end = -1

outlier_threshold = 3

# "Choose all data, where the trip length is less than 3 standard deviations away from the mean"
# This is to remove outliers. Otherwise, our plots would look very squished (since there are some
# VERRRRRY long taxi trips in the dataset)
df_trimmed = df_tr[df_tr["LEN"] < mean + outlier_threshold * std]

# Because our y-values only take on multiples of 15, we want just enough buckets in a histogram
# such that each buckets counts one value's frequency. (e.x. one bucket counts how many 15s trips, 
# how many 30s trips, etc. )
buckets = (int(mean + outlier_threshold * std) // 15)

print(f"Using: {len(df_trimmed)}/{len(df_tr)}")

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(18,14))

# Now, we visualize some features that we think might be useful
for idx, v in enumerate(["YR", "MON", "DAY", "HR", "WK", "ORIGIN_STAND"]):
  # idx // 3 = row, idx % 3 = column
  ax = axs[idx // 3, idx % 3]
  
  # Remove any rows with invalid values
  df_subset = df_trimmed.dropna(subset=v)
  
  # Create a histogram. Look up the documentation for more details
  ax.hist2d(df_subset[v][:end], df_subset["LEN"][:end], cmap="CMRmap", bins=(120,buckets))
  
  # Some stylistic things to make the graphs look nice
  ax.set_xlim(ax.get_xlim()[0] - 1, ax.get_xlim()[1] + 1)
  ax.set_facecolor("black")
  ax.set_ylabel("seconds", fontsize=18)
  ax.set_title(f"Feature: {v}", fontsize=20)


In [ ]:
plt.figure(figsize=(10,10))
for v in [0, 5, 11, 17, 23]:
  # Filter data where the HR matches v
  hourly_data = df_trimmed[df_trimmed["HR"] == v]["LEN"]
  histogram, bin_boundary = np.histogram(hourly_data, bins=buckets)
  histogram = histogram / len(hourly_data)
  # The center is the left_bound and right_bound of a bucket
  bin_centers = [(bin_boundary[i] + bin_boundary[i + 1]) / 2 for i in range(buckets)]
  plt.plot(bin_centers, histogram, label=f"HR={v}")
plt.legend();